In [69]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [70]:
years = [2017,2018,2019,2020,2021]

In [71]:
categories = ['top-gc-riders', 'top-sprinters', 'top-time-trial-riders', 'best-classic-riders']

In [72]:
def chunks(lst, n, start):
    """Yield successive n-sized chunks from lst."""
    for i in range(start, len(lst), n):
        return lst[i:i + n]

In [73]:
def clean_scrape_profile(row, category, year):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di['name'] = name
    di['team'] = team
    di['points'] = points
    di['ranking'] = ranking
    di['type'] = category
    di['year'] = year
    return di

In [74]:
def clean_scrape_performance(row, year):
    di = {}
    name_s = row[1].text.strip()
    name_ls = name_s.split()
    name = ' '.join([i.capitalize() for i in name_ls])
    gc_result = row[2].text
    giro_w = row[3].text
    stage_result = row[4].text
    stage_wins = row[5].text
    top10 = row[5].text
    total = row[7].text
    di['name'] = name
    di['gc_result'] = gc_result
    di['giro_w'] = giro_w
    di['stage_result'] = stage_result
    di['stage_wins'] = stage_wins
    di['top10'] = top10
    di['giro_total'] = total
    di['year'] = year
    return di

In [75]:
def clean_scrape_teams(row, year):
    di = {}
    team = row[1].text.lstrip()
    division = row[2].text
    points = row[3].text
    di['team'] = team
    di['team_class'] = division
    di['team_points'] = points
    di['year'] = year
    return di

In [76]:
def clean_age(row, year):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    age = row[3].text
    di['name'] = name
    di['team'] = team
    di['age'] = age[0:2]
    di['year'] = year
    return di

In [77]:
master_df = pd.DataFrame(columns=['name','team','points','ranking','type','year'])

In [78]:
all_riders = []

In [79]:
def get_age():
    years = [2017,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/giro-d-italia/{year}/gc/startlist/youngest-competitors'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 4)
            ls = []
            for chunk in full_list_chunks:
                try:
                    di = clean_age(chunks(td, 4, chunk), year)
                    ls.append(di)
                except IndexError:
                    continue
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_age.csv', index=False)
get_age()

In [80]:
def get_teams():
    years = [2017,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/giro-d-italia/{year}/gc/startlist/teams-ranked'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 4)
            ls = []
            for chunk in full_list_chunks:
                try:
                    di = clean_scrape_teams(chunks(td, 4, chunk), year)
                    ls.append(di)
                except IndexError:
                    continue
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_teams.csv', index=False)
get_teams()

In [81]:
def get_performance():
    years = [2017,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/giro-d-italia/{year}/gc/startlist/previous-performance'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 9)
            ls = []
            for chunk in full_list_chunks:
                di = clean_scrape_performance(chunks(td, 9, chunk), year)
                ls.append(di)
            sub_df = pd.DataFrame.from_dict(ls)
            master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_performance.csv', index=False)
get_performance()

In [82]:
def get_profile():
    categories = ['top-gc-riders', 'top-sprinters', 'top-time-trial-riders', 'best-classic-riders']
    years = [2017,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
        for category in categories:
            url = f'https://www.procyclingstats.com/race.php?missing=0&filter=Filter&id1=giro-d-italia&id2={year}&id3=gc&id4=startlist&id5={category}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 5)
            ls = []
            for chunk in full_list_chunks:
                di = clean_scrape_profile(chunks(td, 5, chunk), category, year)
                ls.append(di)
            sub_df = pd.DataFrame.from_dict(ls)
            master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_profile.csv', index=False)
get_profile()
        
        

In [83]:
profile = pd.read_csv('data_profile.csv').drop_duplicates()
age = pd.read_csv('data_age.csv').drop_duplicates()
performance = pd.read_csv('data_performance.csv').drop_duplicates()
teams = pd.read_csv('data_teams.csv').drop_duplicates()

In [84]:
age

,name,team,age,year
0,Albanese Vincenzo,Bardiani - CSF,20,2017
2,Martínez Daniel Felipe,Wilier Triestina - Selle Italia,21,2017
5,Pedersen Mads,Trek - Segafredo,21,2017
9,De Plus Laurens,Quick-Step Floors,21,2017
14,Schlegel Michal,CCC Sprandi Polkowice,21,2017
...,...,...,...,...
82520,Richeze Maximiliano,UAE Team Emirates,38,2021
82701,Visconti Giovanni,Bardiani-CSF-Faizanè,38,2021
82883,Keisse Iljo,Deceuninck - Quick Step,38,2021
83066,Pozzovivo Domenico,Team Qhubeka ASSOS,38,2021


In [94]:
table2 = pd.merge(profile, age,  how='outer', left_on=['name','year', 'team'], right_on = ['name','year', 'team'])

In [95]:
#table3 = pd.merge(table2, performance, how='outer', left_on=['name','year'], right_on = ['name','year'])

In [96]:
table4 = pd.merge(table2, teams, how='outer', left_on=['team','year'], right_on = ['team','year']).drop(columns=['ranking', 'team_class'])

In [97]:
table4.head()

,name,team,points,type,year,age,team_points
0,Quintana Nairo,Movistar Team,1550.0,top-gc-riders,2017,27,3999
1,Quintana Nairo,Movistar Team,34.6,top-sprinters,2017,27,3999
2,Quintana Nairo,Movistar Team,358.0,top-time-trial-riders,2017,27,3999
3,Izagirre Gorka,Movistar Team,267.0,top-gc-riders,2017,29,3999
4,Izagirre Gorka,Movistar Team,39.3,top-sprinters,2017,29,3999


In [89]:
sub = table4[['name', 'type', 'points', 'year']]
sb = pd.pivot_table(sub, index=['name', 'year'], columns='type', values='points').reset_index()

In [90]:
table5 = pd.merge(table4, sb, how='outer', left_on=['name','year'], right_on = ['name','year']).drop(columns=['type', 'points']).drop_duplicates()

In [91]:
#table5.sort_values('name').head(30)
table5[table5['name']=="Dumoulin Tom"]

,name,team,year,age,gc_result,giro_w,stage_result,stage_wins,top10,giro_total,team_points,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders
139,Dumoulin Tom,Team Sunweb,2017,26.0,1,1,1.0,4,4,1435.0,2929.0,NaN,425.0,49.0,1212.0
499,Dumoulin Tom,Team Sunweb,2018,27.0,1,1,1.0,4,4,1435.0,2736.0,235.0,669.0,NaN,716.0
910,Dumoulin Tom,Team Sunweb,2019,28.0,1,1,1.0,4,4,1435.0,3763.0,197.0,717.0,32.2,680.0


In [92]:
from sklearn.preprocessing import OneHotEncoder

In [93]:
OneHotEncoder().fit_transform(table4[['type']], sparse=False)

TypeError: fit_transform() got an unexpected keyword argument 'sparse'

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
td = soup.find_all('td')

750

In [ ]:
int(len(td)/5)

150

In [ ]:
full_list_chunks = np.arange(0,len(td), 5)


In [ ]:
def chunks(lst, n, start):
    """Yield successive n-sized chunks from lst."""
    for i in range(start, len(lst), n):
        return lst[i:i + n]

In [ ]:
def clean_scrape(row):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di['name'] = name
    di['team'] = team
    di['points'] = points
    di['ranking'] = ranking
    di['type'] = categories
    di['year'] = '2021'
    return di

In [ ]:
ls = []
for chunk in full_list_chunks:
    di = clean_scrape(chunks(td, 5, chunk))
    ls.append(di)

In [ ]:
pd.DataFrame.from_dict(ls)

,name,team,points,ranking,type,year
0,Evenepoel Remco,Deceuninck - Quick Step,700,4,top-gc-riders,2021
1,Ulissi Diego,UAE Team Emirates,566,8,top-gc-riders,2021
2,Landa Mikel,Bahrain - Victorious,528,11,top-gc-riders,2021
3,Yates Simon,Team BikeExchange,528,10,top-gc-riders,2021
4,Hindley Jai,Team DSM,516,14,top-gc-riders,2021
...,...,...,...,...,...,...
145,Covi Alessandro,UAE Team Emirates,5,846,top-gc-riders,2021
146,Duchesne Antoine,Groupama - FDJ,5,885,top-gc-riders,2021
147,Petilli Simone,Intermarché - Wanty - Gobert Matériaux,5,871,top-gc-riders,2021
148,Nizzolo Giacomo,Team Qhubeka ASSOS,5,822,top-gc-riders,2021


In [ ]:
#row = chunks(td, 5, 0)
row = chunks(td,5,5)
row

[<td>2</td>,
 <td><span class="flag it"></span> <a href="rider/diego-ulissi"><span class="uppercase">Ulissi</span> Diego</a></td>,
 <td>UAE Team Emirates</td>,
 <td>566</td>,
 <td>8</td>]

In [ ]:
clean_scrape(row)

{' Ulissi Diego': ['UAE Team Emirates', '566', '8']}

In [ ]:
di = {}
#di2 = {}
def clean_scrape(row):
    name = row[1].text
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di[name] = [team, points, ranking]
    return di

SyntaxError: 'return' outside function (1095910690.py, line 9)

In [ ]:
[di,di2]

[{}, {' Evenepoel Remco': ['Deceuninck - Quick Step', '700', '4']}]

In [ ]:
pd.DataFrame.from_dict(di).T

,0,1,2
Evenepoel Remco,Deceuninck - Quick Step,700,4
